In [25]:
# Sam Brown
# sam_brown@mines.edu
# June 18
# Goal: Neural Net for predicting time until next event

# Directory
import sys
sys.path.append("/Users/sambrown04/Documents/SURF/whillans-surf/notebooks/SURF")

# Imports
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("/Users/sambrown04/Documents/SURF/Preproc_data/averages_events_2011-13.csv")
df = df.iloc[1:-1] # account for shift to get mins until
df.head()

,pre_slip_area,total_delta,start_time,slip_severity,tide_height,tide_change,form_factor,mins_since,mins_until,h_event
1,126.125599,0.312254,2011-01-11 19:10:00,6.707494e-07,-19.514214,-0.246130,1.647468,665.0,815.0,0
2,138.865041,0.334800,2011-01-12 08:45:00,6.132234e-07,19.337788,-0.079343,1.702534,815.0,760.0,1
3,90.950510,0.303650,2011-01-12 21:25:00,9.796908e-07,-53.653778,-0.151990,1.702534,760.0,790.0,0
4,128.884896,0.324098,2011-01-13 10:35:00,7.178703e-07,25.038245,-0.115554,2.101911,790.0,1385.0,1
5,120.203366,0.473986,2011-01-14 09:40:00,1.042288e-06,53.094581,-0.159486,2.778525,1385.0,605.0,1


In [27]:
# Define features and target
X = df[['mins_since', 'tide_change', 'tide_height', 'h_event', 'form_factor', 'slip_severity', 'total_delta', 'pre_slip_area']]
y = df['mins_until'].values.reshape(-1,1)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = .2, random_state = 42)

# Standardize
x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))


# Pytorch tensors 
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

In [29]:
# Neural Net
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16,1)

    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x) # Regressing
        return x

In [31]:
model = Net()
criterion = nn.MSELoss() # Loss for regression
optimizer = optim.Adam(model.parameters(), lr = .01)

# Training loop
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad() # Clears grad

    # Predictions and loss
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backprop
    loss.backward()

    # Update params
    optimizer.step()
    
    if (epoch+1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [20/200], Loss: 0.8464
Epoch [40/200], Loss: 0.7180
Epoch [60/200], Loss: 0.6324
Epoch [80/200], Loss: 0.5837
Epoch [100/200], Loss: 0.5465
Epoch [120/200], Loss: 0.5235
Epoch [140/200], Loss: 0.5068
Epoch [160/200], Loss: 0.4894
Epoch [180/200], Loss: 0.4722
Epoch [200/200], Loss: 0.4625


In [33]:
# Eval
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor)
    y_pred = y_scaler.inverse_transform(y_pred_scaled.numpy())
    y_test_orig = y_scaler.inverse_transform(y_test_tensor.numpy())

# predictions vs actual
print("\nPredictions vs Actual:")
for pred, actual in zip(y_pred.flatten(), y_test_orig.flatten()):
    print(f"Predicted: {pred:.2f}, Actual: {actual:.2f}")


Predictions vs Actual:
Predicted: 1332.87, Actual: 1470.00
Predicted: 910.41, Actual: 750.00
Predicted: 1070.01, Actual: 870.75
Predicted: 958.19, Actual: 660.00
Predicted: 1352.81, Actual: 1319.25
Predicted: 680.28, Actual: 515.00
Predicted: 1166.99, Actual: 1475.75
Predicted: 1271.58, Actual: 1395.00
Predicted: 564.14, Actual: 606.50
Predicted: 409.05, Actual: 633.25
Predicted: 911.76, Actual: 776.50
Predicted: 699.70, Actual: 900.50
Predicted: 968.89, Actual: 824.25
Predicted: 907.64, Actual: 1406.00
Predicted: 962.44, Actual: 930.00
Predicted: 832.87, Actual: 769.50
Predicted: 833.30, Actual: 757.00
Predicted: 1230.48, Actual: 1443.75
Predicted: 688.23, Actual: 495.00
Predicted: 886.02, Actual: 1270.00
Predicted: 701.77, Actual: 928.75
Predicted: 1241.16, Actual: 1510.00
Predicted: 743.26, Actual: 625.00
Predicted: 859.02, Actual: 787.50
Predicted: 776.21, Actual: 760.00
Predicted: 1002.47, Actual: 1395.00
Predicted: 1358.38, Actual: 1030.00
Predicted: 773.88, Actual: 598.50
Predi

In [35]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test_orig, y_pred)
print(r2)

0.08738523721694946
